In [ ]:
import psycopg2
import os

First lets connect ourselves with our POSTGRES db. The credentials are also copied in the `docker-compose.yml`.
So before you are exec the next cell, please make sure that docker is installed on your machine. :)

In [ ]:
# Connect to the database by exec docker compose in your terminal. This exec. a terminal command using Python
os.system("""
docker compose down;
docker compose up -d --build db
""")

When the cell returns 0, it means that everything was done correctly.

In [ ]:
try:
    # Connect to your PostgreSQL database. Here are the credentials written so the machine knows where connect to.
    connection = psycopg2.connect(
        user="user",
        password="user_pw",
        host="localhost",
        port="5432",
        database="search_engine_db"
    )

    # Create a cursor object. We need this to execute queries.
    cursor = connection.cursor()

    # Print PostgreSQL version just as a sanity check.
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    # Example query: Create a table
    create_search_index_table = '''
        CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
        
        CREATE TABLE IF NOT EXISTS documents (
        id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
        url TEXT,
        title TEXT,
	    headings TEXT[],
        page_text TEXT,
        keywords TEXT[],
        accessed_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        internal_links TEXT[],
        external_links TEXT[],
        enc_text TEXT[],
        summary TEXT);   
    );
    '''
    
    # Executes a query statement
    cursor.execute(create_search_index_table)
    
    # Commit the transaction to save changes. Should always be called after `cursor.execute(...)`
    connection.commit()
    print("Table created successfully")

    # Example query: Insert data
    insert_query_example = "INSERT INTO documents (url, content) VALUES (%s, %s)"
    
    cursor.execute(insert_query_example, ('testurl', 'This is a nice content page!'))
    connection.commit()
    print("Data inserted successfully")

    # Example query: Retrieve data
    cursor.execute("SELECT * FROM documents")
    records = cursor.fetchall()
    print("Data retrieved successfully:")
    for row in records:
        print(row)
    
    # To finish the connection with our db after you interact with it.
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

When you see `PostgreSQL connection is closed`, you know that everything was done correctly. Now you can peak into the database, with e.g. DBeaver and see how the data looks like.

## CAUTION
exec this table to delete all entries from the database


In [ ]:
# CAUTION: exec this table to delete all entries from the database

try:
    # Connect to your PostgreSQL database. Here are the credentials written so the machine knows where connect to.
    connection = psycopg2.connect(
        user="user",
        password="user_pw",
        host="localhost",
        port="5432",
        database="search_engine_db"
    )

    # Create a cursor object. We need this to execute queries.
    cursor = connection.cursor()

    # Delete all entries
    cursor.execute("DELETE FROM documents")
    connection.commit()
    print("Entries deleted successfully")
    
    cursor.execute("SELECT * FROM documents")
    records = cursor.fetchall()
    print("Data retrieved successfully:")
    if len(records) == 0:
        print("No entries found")
    
    # To finish the connection with our db after you interact with it.
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

Finally, you should shut down the container and kill it.

In [ ]:
os.system("""
docker compose down;
""")